In [22]:
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
%matplotlib inline

df=pd.read_excel('Election_Data.xlsx',sheet_name='Election_Dataset_Two Classes')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True,drop=True)
df =pd.get_dummies(df, columns=['gender','vote'],drop_first=True)
df

,age,economic.cond.national,economic.cond.household,Blair,Hague,Europe,political.knowledge,gender_male,vote_Labour
0,43,3,3,4,1,2,2,0,1
1,36,4,4,4,4,5,2,1,1
2,35,4,4,5,2,3,2,1,1
3,24,4,2,2,1,4,0,0,1
4,41,2,2,1,1,6,2,1,1
...,...,...,...,...,...,...,...,...,...
1512,67,5,3,2,4,11,3,1,0
1513,73,2,2,4,4,8,2,1,0
1514,37,3,3,5,4,2,2,1,1
1515,61,3,3,1,4,11,2,1,0


In [24]:
X = df.drop("vote_Labour" , axis=1)
y = df['vote_Labour']

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=1, stratify=y)

In [26]:
X_train

,age,economic.cond.national,economic.cond.household,Blair,Hague,Europe,political.knowledge,gender_male
532,71,3,3,4,2,11,0,0
706,57,4,5,4,1,3,2,0
1140,24,3,4,2,4,11,0,1
1077,43,4,4,2,3,5,0,1
954,37,3,2,4,2,7,2,1
...,...,...,...,...,...,...,...,...
512,80,2,2,2,4,7,2,1
825,27,3,3,2,4,5,2,0
348,56,4,2,5,2,1,2,0
1241,75,4,3,4,5,7,3,0


In [27]:
from sklearn.ensemble import AdaBoostClassifier
#Build ADA Model
clf = AdaBoostClassifier(random_state=1)
model=clf.fit(X_train,y_train)

In [28]:
model.score(X_train, y_train)

C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


0.8397737983034873

In [29]:
model.score(X_test, y_test)

C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


0.8355263157894737

In [30]:
pred_train_prob = model.predict_proba(X_train)
pred_test_prob = model.predict_proba(X_test)
pred_train_prob

C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(
C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


array([[0.48997962, 0.51002038],
       [0.4783442 , 0.5216558 ],
       [0.51155709, 0.48844291],
       ...,
       [0.46075784, 0.53924216],
       [0.50627138, 0.49372862],
       [0.50168899, 0.49831101]])

In [31]:
pred_train_prob = pred_train_prob[:,1]

In [32]:
from sklearn.metrics import accuracy_score,f1_score,recall_score, precision_score


def GetCustomData(custom_prob):
    custom_cutoff_data=[]

    for i in range(0,len(pred_train_prob)):  #2333
        if pred_train_prob[i] >custom_prob:
            a= 1
        else:
            a=0
        custom_cutoff_data.append(a)
    return custom_cutoff_data

In [33]:
thresh=[]
acc=[]
f1=[]
rec=[]
pre=[]

In [34]:
for j in np.arange(0.1,1,0.1):
    result = GetCustomData(j)
    thresh.append(j)
    acc.append(accuracy_score(y_train,result))
    f1.append(f1_score(y_train,result))
    rec.append(recall_score(y_train,result))
    pre.append(precision_score(y_train,result))


C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jitu\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is i

In [35]:
dat = pd.DataFrame({'Threshold': thresh,
             'Accuracy': acc,
             'f1': f1,
             'Recall': rec,
             'Precision':pre})
dat

,Threshold,Accuracy,f1,Recall,Precision
0,0.1,0.696513,0.821111,1.000000,0.696513
1,0.2,0.696513,0.821111,1.000000,0.696513
2,0.3,0.696513,0.821111,1.000000,0.696513
3,0.4,0.698398,0.822024,1.000000,0.697828
4,0.5,0.839774,0.886515,0.898512,0.874835
5,0.6,0.303487,0.000000,0.000000,0.000000
6,0.7,0.303487,0.000000,0.000000,0.000000
7,0.8,0.303487,0.000000,0.000000,0.000000
8,0.9,0.303487,0.000000,0.000000,0.000000


In [36]:
dat[dat['f1']==dat['f1'].max()]

,Threshold,Accuracy,f1,Recall,Precision
4,0.5,0.839774,0.886515,0.898512,0.874835


In [37]:
dat[dat['Accuracy']==dat['Accuracy'].max()]

,Threshold,Accuracy,f1,Recall,Precision
4,0.5,0.839774,0.886515,0.898512,0.874835
